### Import Modules

In [1]:
from config_VVV_IRAC import *
import matplotlib.pyplot as plt

import os, glob
from astropy.io import fits
from astropy.wcs import WCS

In [2]:
# import matplotlib.pyplot as plt
# import os, copy, glob
# import reproject
# from astropy.io import fits
# from astropy.wcs import WCS

In [3]:
# # Added Modules for Registration
# from astropy.io import fits
# from utils import ProjectionVisualizer

In [2]:
# from config import *
# from config_VVV_IRAC import *
# from utils import GridVisualizer
from dataset import FitsArrayConverter
from dataset import ImageStacker, ImageAugmentation

#### Check the Pixel Scale in the Header

In [3]:
def get_image(filename, header=False):
    file = fits.open(filename)
    image = file[-1].data
    if header:
        header = file[-1].header
        return image, header
    else:
        return image

In [117]:
# 01) images_added/XXX_aligned
# 02) images_added/XXX_aligned_cropped
# 03) images_added/XXX_aligned_sparse_cropped
# vvv_filename = os.path.join(root_dir, 'images_added', 'vvv_raw', 'gc_vvv_H.fits.fz')
# irac_filename = os.path.join(root_dir, 'images_added', 'spitzer_raw', 'gc_irac_ch1_jun06_eq_withsub.fits')
# ukidss_filename = os.path.join(root_dir, 'images', 'ukidss', 'rectified_244532_H.fits')

In [6]:
# 03_01) images_added/XXX_aligned_sstgc_target (Cropping for SSTGC Targets)
vvv_filename = os.path.join(root_dir, 'images_added', 'vvv_raw_sstgc_target', 'gc_vvv_H_799887.fits.fz')
irac_filename = os.path.join(root_dir, 'images_added', 'spitzer_raw', 'gc_irac_ch1_jun06_eq_withsub.fits')
ukidss_filename = os.path.join(root_dir, 'images', 'ukidss', 'rectified_244532_H.fits')

In [7]:
### VVV
vvv_image, vvv_header = get_image(vvv_filename, header=True)

cd1_1 = vvv_header.get('CD1_1', 0.0)
cd1_2 = vvv_header.get('CD1_2', 0.0)
cd2_1 = vvv_header.get('CD2_1', 0.0)
cd2_2 = vvv_header.get('CD2_2', 0.0)

# Pixel scale in degrees/pixel
# pixel_scale_deg = np.sqrt(cd1_1**2 + cd1_2**2 + cd2_1**2 + cd2_2**2) / np.sqrt(2) 
pixel_scale_deg = np.sqrt(cd1_1**2 + cd2_1**2)   # along x-axis

# Convert to arcsec/pixel
pixel_scale_vvv = pixel_scale_deg * 3600.0  # 1 degree = 3600 arcsec
print(f'#-- Calculated Pixel Scale: {pixel_scale_vvv:.2f} [arcsec/pix]')
print(f'(with using values in CD Matrix; {cd1_1:.2e} {cd1_2:.2e} {cd2_1:.2e} {cd2_2:.2e})')

#-- Calculated Pixel Scale: 0.34 [arcsec/pix]
(with using values in CD Matrix; 8.12e-05 4.89e-05 -4.89e-05 8.12e-05)


In [ ]:
### IRAC
irac_image, irac_header = get_image(irac_filename, header=True)

# Pixel scale in degrees/pixel
pixel_scale_deg = irac_header.get('CDELT2', 0.0)   # along x-axis

# Convert to arcsec/pixel
pixel_scale_irac = pixel_scale_deg * 3600.0  # 1 degree = 3600 arcsec
print(f'#-- Calculated Pixel Scale: {pixel_scale_irac:.2f} [arcsec/pix]')

#-- Calculated Pixel Scale: 1.00 [arcsec/pix]


In [ ]:
### UKIDSS
ukidss_image, ukidss_header = get_image(ukidss_filename, header=True)

cd1_1 = ukidss_header.get('CD1_1', 0.0)
cd1_2 = ukidss_header.get('CD1_2', 0.0)
cd2_1 = ukidss_header.get('CD2_1', 0.0)
cd2_2 = ukidss_header.get('CD2_2', 0.0)
pix_size = ukidss_header.get('PIXLSIZE', 0.0)

# Pixel scale in degrees/pixel
pixel_scale_deg_x = np.sqrt(cd1_1**2 + cd2_1**2)   # along x-axis
pixel_scale_deg_y = np.sqrt(cd1_2**2 + cd2_2**2)   # along y-axis
pixel_scale_deg = np.sqrt(cd1_1**2 + cd1_2**2 + cd2_1**2 + cd2_2**2) / np.sqrt(2)

# Convert to arcsec/pixel
pixel_scale_ukidss = pixel_scale_deg * 3600.0  # 1 degree = 3600 arcsec
print(f'#-- Calculated Pixel Scale: {pixel_scale_ukidss:.2f} [arcsec/pix]')
print(f'(with using values in CD Matrix; {cd1_1:.2e} {cd1_2:.2e} {cd2_1:.2e} {cd2_2:.2e})')
print()
print(f'#-- Recorded Pixel Scale: {pix_size:.2f} [arcsec/pix]')

#-- Calculated Pixel Scale: 0.20 [arcsec/pix]
(with using values in CD Matrix; -5.57e-05 -3.81e-07 -3.77e-07 5.57e-05)

#-- Recorded Pixel Scale: 0.40 [arcsec/pix]


In [ ]:
print(f'For 1[arcmin] x 1[arcmin] FOV...')
print(f'- VVV    : ~ {int(60./round(pixel_scale_vvv, 2))} [pix]')
print(f'- IRAC   : ~ {int(60./round(pixel_scale_irac, 2))} [pix]')
print(f'- UKIDSS : ~ {int(60./round(pixel_scale_ukidss, 2))} [pix]')

For 1[arcmin] x 1[arcmin] FOV...
- VVV    : ~ 176 [pix]
- IRAC   : ~ 60 [pix]
- UKIDSS : ~ 300 [pix]


### Configuration

In [4]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    device = "cuda:0"

In [5]:
# Editable Options
channels_order = ['J', 'H', 'K', 'ch2', 'ch3', 'ch4']
# channels_order = ['H', 'K', 'ch2']
extract_fits_patch = False
convert_fits_to_npy = False
verbose_fits_to_npy = False
save_projection_visualize = False
confirm_saved_npy = False
match_output_channel_num = True   ###

# Channel Number (Derived)
input_nc = len(channels_order)  # 6
if match_output_channel_num:
    output_nc = input_nc
else:
    output_nc = 1

In [ ]:
# 01) images_added/XXX_aligned
# stacking_size = 174
# save_dir = root_dir+'/data_added/registrated_gc_180x180_extraction/merged'
# add_suffix='_to_6_channels_registration_VVV'   # for saving files

In [ ]:
# 02) images_added/XXX_aligned_cropped
# stacking_size = 174
# save_dir = root_dir+'/data_added/registrated_cropped_gc_180x180_extraction/merged'
# add_suffix='_to_6_channels_registration_cropped_VVV'   # for saving files

In [15]:
# 03) images_added/XXX_aligned_sparse_cropped
# stacking_size = 174
# save_dir = root_dir+'/data_added/registrated_sparse_cropped_gc_180x180_extraction/merged'
# add_suffix='_to_6_channels_registration_sparse_cropped_VVV'   # for saving files

In [10]:
# 03_01) images_added/XXX_aligned_sstgc_target (Cropping for SSTGC Targets)
# stacking_size = 174
# save_dir = root_dir+'/data_added/registrated_sparse_cropped_gc_180x180_extraction/sstgc_targets/merged'
# add_suffix='_to_6_channels_registration_sstgc_target_cropped_VVV'   # for saving files

In [20]:
# 04) images_added/XXX_aligned_sparse_cropped_3ch
stacking_size = 174
save_dir = root_dir+'/data_added/registrated_sparse_cropped_gc_180x180_extraction/merged_3ch'
add_suffix='_to_3_channels_registration_sparse_cropped_VVV'   # for saving files

In [7]:
print(f'Base Directory: {root_dir}')
print('Device:', device)
print(f"Number of CPU Cores: {num_cores}")

Base Directory: /home/jiwon/UKIDSSxIRAC
Device: cuda:0
Number of CPU Cores: 32


### Make a Filelist for Whole Image

In [ ]:
# 01) images_added/XXX_aligned
# image_dir1 = os.path.join(root_dir,'images_added',survey1+'_aligned')
# image_dir2 = os.path.join(root_dir,'images_added',survey2+'_aligned')

In [ ]:
# 02) images_added/XXX_aligned_cropped
# image_dir1 = os.path.join(root_dir,'images_added',survey1+'_aligned_cropped')
# image_dir2 = os.path.join(root_dir,'images_added',survey2+'_aligned_cropped')

In [126]:
# 03) images_added/XXX_aligned_sparse_cropped
# image_dir1 = os.path.join(root_dir,'images_added',survey1+'_aligned_sparse_cropped')
# image_dir2 = os.path.join(root_dir,'images_added',survey2+'_aligned_sparse_cropped')

In [15]:
# 03_01) images_added/XXX_aligned_sstgc_target (Cropping for SSTGC Targets)
# image_dir1 = os.path.join(root_dir,'images_added',survey1+'_aligned_sstgc_target')
# image_dir2 = os.path.join(root_dir,'images_added',survey2+'_aligned_sstgc_target')

In [9]:
# 04) images_added/XXX_aligned_sparse_cropped_3ch
image_dir1 = os.path.join(root_dir,'images_added',survey1+'_aligned_sparse_cropped')
image_dir2 = os.path.join(root_dir,'images_added',survey2+'_aligned_sparse_cropped')

In [10]:
print(image_dir1, image_dir2)

/home/jiwon/UKIDSSxIRAC/images_added/vvv_aligned_sparse_cropped /home/jiwon/UKIDSSxIRAC/images_added/spitzer_aligned_sparse_cropped


In [11]:
# List Up the Files
imagelist = []
ref_header_filename = None

imagelist1 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(image_dir1, f'gc_*.fits'))])
imagelist2 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(image_dir2, f'gc_*.fits'))])
imagelist= imagelist1+imagelist2
# print('\n'.join(imagelist), '\n')

# Find the Reference Coord File
try:
    from config_VVV_IRAC import ref_coord_band
    ref_header_filename = imagelist[next(i for i, file in enumerate(imagelist) if f'_{ref_coord_band}' in file)]
except StopIteration:
    ref_header_filename = None
    print(f'No reference coordinate file found!')

In [12]:
print('\n'.join(imagelist), '\n')

/home/jiwon/UKIDSSxIRAC/images_added/vvv_aligned_sparse_cropped/gc_vvv_H.fits
/home/jiwon/UKIDSSxIRAC/images_added/vvv_aligned_sparse_cropped/gc_vvv_J.fits
/home/jiwon/UKIDSSxIRAC/images_added/vvv_aligned_sparse_cropped/gc_vvv_K.fits
/home/jiwon/UKIDSSxIRAC/images_added/spitzer_aligned_sparse_cropped/gc_irac_ch1.fits
/home/jiwon/UKIDSSxIRAC/images_added/spitzer_aligned_sparse_cropped/gc_irac_ch2.fits
/home/jiwon/UKIDSSxIRAC/images_added/spitzer_aligned_sparse_cropped/gc_irac_ch3.fits
/home/jiwon/UKIDSSxIRAC/images_added/spitzer_aligned_sparse_cropped/gc_irac_ch4.fits 



### Data Preparation

#### Settings

In [ ]:
# 01) images_added/XXX_aligned
# dirname = f'registrated_gc_180x180_extraction'

In [ ]:
# 02) images_added/XXX_aligned_cropped
# dirname = f'registrated_cropped_gc_180x180_extraction'

In [3]:
# 03) images_added/XXX_aligned_sparse_cropped
dirname = f'registrated_sparse_cropped_gc_180x180_extraction'

In [19]:
# 03_01) images_added/XXX_aligned_sparse_cropped (Cropping for SSTGC Targets)
dirname = f'registrated_sparse_cropped_gc_180x180_extraction/sstgc_targets/'

In [18]:
# 04) images_added/XXX_aligned_sparse_cropped_3ch
dirname = f'registrated_sparse_cropped_gc_180x180_extraction/'

In [21]:
# Make Data Directory
if not os.path.exists(root_dir+'/data_added/'+dirname+'/'+survey1):
    print(f'\n=== Making "data_added/{dirname}" Directory ===\n')
    os.makedirs(root_dir+f'/data_added/{dirname}')
    os.makedirs(root_dir+f'/data_added/{dirname}/'+survey1)
    os.makedirs(root_dir+f'/data_added/{dirname}/'+survey2)

#### Load Source Information

In [15]:
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u

class ConvertCoord():
    def __init__(self):
        pass
    
    def load_dataframe(self, filename=None, sep='\t', header=None, names=None, rename=False,
                       dataframe=None, verbose=False):
        if (filename is None) and (dataframe is None):
            raise ValueError("Either a filename or a DataFrame must be provided.")
        if (filename is not None) and (dataframe is not None):
            raise ValueError("Both filename and DataFrame cannot be provided simultaneously.")

        if dataframe is not None:
            df = dataframe
        if filename is not None:
            if header is None:
                names = ['RA', 'Dec', 'ID']
            df = pd.read_csv(filename, sep=sep, header=header, names=names, dtype={'ID': str})
            if header is not None and rename:
                df = self.rename_columns(df, rename_values=None)
        if verbose:
            print(df.head())
        return df

    def rename_columns(self, df, rename_values=None):
        if rename_values is None:
            rename_values = ['RA', 'Dec', 'ID']
        input_keys = df.columns.tolist()
        if len(input_keys) != len(rename_values):
            raise ValueError(f"Expected {len(rename_values)} columns, but got {len(input_keys)}.")
        rename_map = dict(zip(input_keys, rename_values))
        print("Before renaming:", df.columns.tolist())
        df = df.rename(columns=rename_map)
        print("After renaming:", df.columns.tolist())
        return df

    def convert_row(self, row):
        coord = SkyCoord(ra=row["RA"], dec=row["Dec"], unit=(u.hourangle, u.deg), frame='icrs')
        return pd.Series({'RA': coord.ra.deg, 'Dec': coord.dec.deg})
    
    def convert_to_deg(self, df, verbose=False):
        df[["RA", "Dec"]] = df.apply(lambda row: self.convert_row(row), axis=1)
        if verbose:
            print('\n', df.head(),'\n')
            print(f"Confirm Data Type - RA: {type(df['RA'][0])}, Dec: {type(df['Dec'][0])}")
        return df

    def add_galactic_coord(self, df, verbose=False):
        coords = SkyCoord(ra=df['RA'].values * u.deg, dec=df['Dec'].values * u.deg, frame='icrs')
        df['GLON'] = coords.galactic.l.deg
        df['GLAT'] = coords.galactic.b.deg
        df['Adjusted_GLON'] = df['GLON'].apply(lambda x: x - 360 if x > 180 else x)

        if verbose:
            print('\n', df.head(),'\n')
        return df

convert_coord = ConvertCoord()

In [16]:
# 01) images_added/XXX_aligned
# 02) images_added/XXX_aligned_cropped
# 03) images_added/XXX_aligned_sparse_cropped
# 04) images_added/XXX_aligned_sparse_cropped_3ch
df_filename = 'targetlist_filtered_sources_with_source_name.txt'
df = convert_coord.load_dataframe(df_filename, sep=',', header=0, names=['RA', 'Dec', 'ID'])
# df = convert_coord.add_galactic_coord(df)

coords_center = SkyCoord(ra=df['RA'].values * u.deg, dec=df['Dec'].values * u.deg, frame='icrs')
df['RA_Dec_String'] = [
    f"{ra_hms}{dec_dms}" for ra_hms, dec_dms in zip(
        coords_center.ra.to_string(unit=u.hour, sep='', precision=1),
        coords_center.dec.to_string(unit=u.deg, sep='', precision=0))]
df['Converted_FileName'] = [
    f"{i+1:04}_{ra_dec}_" for i, ra_dec in enumerate(df['RA_Dec_String'])]

print(f'Shape of the Data Frame: {df.shape}')
df.head()

Shape of the Data Frame: (2163, 5)


,RA,Dec,ID,RA_Dec_String,Converted_FileName
0,265.176917,-29.433111,0012590,174042.5-292600,0001_174042.5-292600_
1,265.184542,-29.634444,0013582,174044.3-293804,0002_174044.3-293804_
2,265.213583,-29.339694,0017377,174051.3-292023,0003_174051.3-292023_
3,265.245833,-29.176111,0022932,174059.0-291034,0004_174059.0-291034_
4,265.264542,-29.099167,0026315,174103.5-290557,0005_174103.5-290557_


In [24]:
# 03_01) images_added/XXX_aligned_sstgc_target (Cropping for SSTGC Targets)
df_sstgc_filename = 'targetlist_with_coord.txt'
df_sstgc = convert_coord.load_dataframe(df_sstgc_filename, sep='\t', header=None, names=['RA', 'Dec', 'ID'])

coords_center = SkyCoord(ra=df_sstgc['RA'], dec=df_sstgc['Dec'], unit=(u.hourangle, u.deg), frame='icrs')
df_sstgc['RA_Dec_String'] = [
    f"{ra_hms}{dec_dms}" for ra_hms, dec_dms in zip(
        coords_center.ra.to_string(unit=u.hour, sep='', precision=1),
        coords_center.dec.to_string(unit=u.deg, sep='', precision=1))]
df_sstgc['Converted_FileName'] = [
    f"{id}_{ra_dec}_" for id, ra_dec in zip(df_sstgc['ID'], df_sstgc['RA_Dec_String'])]

df_sstgc['RA'] = coords_center.ra.deg
df_sstgc['Dec'] = coords_center.dec.deg

print(f'Shape of the Data Frame: {df_sstgc.shape}')
df_sstgc.head()

Shape of the Data Frame: (107, 5)


,RA,Dec,ID,RA_Dec_String,Converted_FileName
0,265.949875,-29.644778,244532,174348.0-293841.2,244532_174348.0-293841.2_
1,265.983250,-29.606222,260956,174356.0-293622.4,260956_174356.0-293622.4_
2,265.988833,-29.611278,263857,174357.3-293640.6,263857_174357.3-293640.6_
3,266.028792,-29.404833,284291,174406.9-292417.4,284291_174406.9-292417.4_
4,266.046667,-29.443861,293528,174411.2-292637.9,293528_174411.2-292637.9_


#### Extract and Save Patch Image

In [17]:
class PatchExtractor():
    def __init__(self):
        pass
    
    def open_fits_file(self, filename):
        file = fits.open(filename)
        image, header = file[-1].data, file[-1].header
        return image, header

    def convert_center_pix(self, wcs, ra, dec):
        x_center, y_center = wcs.world_to_pixel_values(ra, dec)
        x_center, y_center = int(x_center), int(y_center)
        return x_center, y_center

    def calculate_extraction_width(self, header, radius_deg):
        if 'CDELT1' and 'CDELT2' in header:
            pixel_scale_ra = abs(header['CDELT1'])
            pixel_scale_dec = abs(header['CDELT2'])
        elif 'CD1_1' in header and 'CD2_1' in header:
            cd1_1 = header['CD1_1']
            cd1_2 = header['CD1_2']
            cd2_1 = header['CD2_1']
            cd2_2 = header['CD2_2']
            pixel_scale_ra = np.sqrt(cd1_1**2 + cd2_1**2)
            pixel_scale_dec = np.sqrt(cd1_2**2 + cd2_2**2)
        else:
            print("Pixel scale information not found in header.")
            pixel_scale_ra = 0.34/3600.
            pixel_scale_dec = 0.34/3600.
        half_width_x = int(radius_deg / pixel_scale_ra) 
        half_width_y = int(radius_deg / pixel_scale_dec)
        return half_width_x, half_width_y

    def get_extraction_point(self, image, x_center, y_center, half_width_x, half_width_y):
        # x_min, x_max = max(0, x_center - half_width_x), min(image.shape[1], x_center + half_width_x)
        # y_min, y_max = max(0, y_center - half_width_y), min(image.shape[0], y_center + half_width_y)
        # return x_min, x_max, y_min, y_max
        image_height, image_width = image.shape

        x_min = x_center - half_width_x
        x_max = x_center + half_width_x
        y_min = y_center - half_width_y
        y_max = y_center + half_width_y

        # Validate patch bounds
        if x_min < 0 or y_min < 0 or x_max > image_width or y_max > image_height:
            return None, None, None, None, False  # Patch is out of bounds
        return x_min, x_max, y_min, y_max, True  # Valid patch
    
    def extract_subimage(self, image, x_min, x_max, y_min, y_max):
        subimage = image[y_min:y_max, x_min:x_max]
        return subimage
    
    def save_patch_as_fits(self, subimage, original_header, x_min, y_min, output_filename, verbose=False):
        new_header = original_header.copy()

        if 'CRPIX1' in new_header and 'CRPIX2' in new_header:
            new_header['CRPIX1'] -= x_min  # Adjust RA reference pixel
            new_header['CRPIX2'] -= y_min  # Adjust Dec reference pixel
        else:
            raise(ValueError("CRPIX1 or CRPIX2 not found in the header."))
        new_header['NAXIS1'] = subimage.shape[1]
        new_header['NAXIS2'] = subimage.shape[0]
        new_header['COMMENT'] = "Extracted subimage with updated WCS."

        hdu = fits.PrimaryHDU(data=subimage, header=new_header)
        hdu.writeto(output_filename, overwrite=True)
        if verbose:
            print(f"Saved FITS file: {output_filename}")

    def calculate_patch_pixel(self, image, header, ra, dec, radius_deg):
        wcs = WCS(header)
        x_center, y_center = wcs.world_to_pixel_values(ra, dec)
        x_center, y_center = int(x_center), int(y_center)

        if 'CDELT1' and 'CDELT2' in header:
            pixel_scale_ra = abs(header['CDELT1'])
            pixel_scale_dec = abs(header['CDELT2'])
        elif 'CD1_1' in header and 'CD2_1' in header:
            cd1_1 = header['CD1_1']
            cd1_2 = header['CD1_2']
            cd2_1 = header['CD2_1']
            cd2_2 = header['CD2_2']
            pixel_scale_ra = np.sqrt(cd1_1**2 + cd2_1**2)
            pixel_scale_dec = np.sqrt(cd1_2**2 + cd2_2**2)
        else:
            print("Pixel scale information not found in header.")
            pixel_scale_ra = 0.34/3600.   # VVV pixel scale = 0.32 [arcsec/pix]
            pixel_scale_dec = 0.34/3600.
        half_width_x = int(radius_deg / pixel_scale_ra) 
        half_width_y = int(radius_deg / pixel_scale_dec)
        image_height, image_width = image.shape

        x_min = x_center - half_width_x
        x_max = x_center + half_width_x
        y_min = y_center - half_width_y
        y_max = y_center + half_width_y

        # Validate patch bounds
        if x_min < 0 or y_min < 0 or x_max > image_width or y_max > image_height:
            return None, None, None, None, False  # Patch is out of bounds
        return x_min, x_max, y_min, y_max, True  # Valid patch

In [ ]:
# 01) images_added/XXX_aligned
# Make Data Directory
if not os.path.exists(os.path.join(root_dir,'images_added',survey1+'_extracted')):
    os.makedirs(os.path.join(root_dir,'images_added',survey1+'_extracted'))
    os.makedirs(os.path.join(root_dir,'images_added',survey2+'_extracted'))

In [83]:
# 02) images_added/XXX_aligned_cropped
# Make Data Directory
if not os.path.exists(os.path.join(root_dir,'images_added',survey1+'_extracted_cropped')):
    os.makedirs(os.path.join(root_dir,'images_added',survey1+'_extracted_cropped'))
    os.makedirs(os.path.join(root_dir,'images_added',survey2+'_extracted_cropped'))

In [134]:
# 03) images_added/XXX_aligned_sparse_cropped
# Make Data Directory
if not os.path.exists(os.path.join(root_dir,'images_added',survey1+'_extracted_sparse_cropped')):
    os.makedirs(os.path.join(root_dir,'images_added',survey1+'_extracted_sparse_cropped'))
    os.makedirs(os.path.join(root_dir,'images_added',survey2+'_extracted_sparse_cropped'))

In [26]:
# 03_01) images_added/XXX_aligned_sstgc_target (Cropping for SSTGC Targets)
# Make Data Directory
if not os.path.exists(os.path.join(root_dir,'images_added',survey1+'_extracted_sstgc_target')):
    os.makedirs(os.path.join(root_dir,'images_added',survey1+'_extracted_sstgc_target'))
    os.makedirs(os.path.join(root_dir,'images_added',survey2+'_extracted_sstgc_target'))

In [ ]:
patch_extractor = PatchExtractor()

ref_image, ref_header = patch_extractor.open_fits_file(ref_header_filename)
wcs = WCS(ref_header)
radius_deg = 0.5 * (1/60.)

half_width_x, half_width_y = patch_extractor.calculate_extraction_width(ref_header, radius_deg)
stacking_size = half_width_x * 2

image_dict = {}
for imagefile in imagelist:
    image, _ = patch_extractor.open_fits_file(imagefile)
    band = os.path.splitext(imagefile)[0].split('_')[-1]
    image_dict[band] = image

valid_image = 0
if True:#extract_fits_patch: 
    for _, row in df.iterrows():
        ra, dec = row["RA"], row["Dec"]
        x_center, y_center = patch_extractor.convert_center_pix(wcs, ra, dec)
        x_min, x_max, y_min, y_max, is_valid = patch_extractor.get_extraction_point(ref_image, x_center, y_center, half_width_x, half_width_y)

        # Check if the patch is valid
        if not is_valid:
            idx = row["Converted_FileName"][0:4]
            print(f"### '{idx}' - Skipping patch for RA: {ra}, Dec: {dec} as it is out of bounds.")
            continue
        valid_image += 1  # Increment valid image count if the patch is valid
        
        for i, band in enumerate(list(image_dict)):
            survey = survey1 if band in filters1 else survey2

            # 01) images_added/XXX_aligned
            # output_filename = os.path.join(root_dir,'images_added',survey+'_extracted', row["Converted_FileName"]+band+'.fits')

            # 02) images_added/XXX_aligned_cropped
            # output_filename = os.path.join(root_dir,'images_added',survey+'_extracted_cropped', row["Converted_FileName"]+band+'.fits')

            # 03) images_added/XXX_aligned_sparse_cropped
            output_filename = os.path.join(root_dir,'images_added',survey+'_extracted_sparse_cropped', row["Converted_FileName"]+band+'.fits')

            subimage = patch_extractor.extract_subimage(image_dict[band], x_min, x_max, y_min, y_max)
            patch_extractor.save_patch_as_fits(subimage, ref_header, x_min, y_min, output_filename)
            
            # plt.figure()
            # plt.imshow(subimage)
            # plt.title(band)

    print(f'The number of saved valid dataset: {valid_image}')
    print()
    print(f'The Shape of SubImage: {np.shape(subimage)}')
    print(f'Stacking Size: {stacking_size}')

In [ ]:
# 03_01) images_added/XXX_aligned_sstgc_target (Cropping for SSTGC Targets)
patch_extractor = PatchExtractor()

ref_image, ref_header = patch_extractor.open_fits_file(ref_header_filename)
wcs = WCS(ref_header)
radius_deg = 0.5 * (1/60.)

half_width_x, half_width_y = patch_extractor.calculate_extraction_width(ref_header, radius_deg)
stacking_size = half_width_x * 2

image_dict = {}
for imagefile in imagelist:
    image, _ = patch_extractor.open_fits_file(imagefile)
    band = os.path.splitext(imagefile)[0].split('_')[-1]
    image_dict[band] = image

valid_image = 0
if True:#extract_fits_patch: 
    for _, row in df_sstgc.iterrows():
        ra, dec = row["RA"], row["Dec"]
        x_center, y_center = patch_extractor.convert_center_pix(wcs, ra, dec)
        x_min, x_max, y_min, y_max, is_valid = patch_extractor.get_extraction_point(ref_image, x_center, y_center, half_width_x, half_width_y)

        # Check if the patch is valid
        if not is_valid:
            sstgc = row["Converted_FileName"][0:6]
            print(f"### '{sstgc}' - Skipping patch for RA: {ra}, Dec: {dec} as it is out of bounds.")
            continue
        valid_image += 1  # Increment valid image count if the patch is valid
        
        for i, band in enumerate(list(image_dict)):
            survey = survey1 if band in filters1 else survey2

            # 03_01) images_added/XXX_aligned_sstgc_target (Cropping for SSTGC Targets)
            output_filename = os.path.join(root_dir,'images_added',survey+'_extracted_sstgc_target', row["Converted_FileName"]+band+'.fits')

            subimage = patch_extractor.extract_subimage(image_dict[band], x_min, x_max, y_min, y_max)
            patch_extractor.save_patch_as_fits(subimage, ref_header, x_min, y_min, output_filename)
            
            # plt.figure()
            # plt.imshow(subimage)
            # plt.title(band)

    print(f'The number of saved valid dataset: {valid_image}')
    print()
    print(f'The Shape of SubImage: {np.shape(subimage)}')
    print(f'Stacking Size: {stacking_size}')

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


### '244532' - Skipping patch for RA: 265.9498749999999, Dec: -29.644777777777776 as it is out of bounds.
### '260956' - Skipping patch for RA: 265.98324999999994, Dec: -29.606222222222225 as it is out of bounds.
### '263857' - Skipping patch for RA: 265.9888333333333, Dec: -29.61127777777778 as it is out of bounds.
### '284291' - Skipping patch for RA: 266.0287916666666, Dec: -29.404833333333332 as it is out of bounds.
### '293528' - Skipping patch for RA: 266.0466666666666, Dec: -29.44386111111111 as it is out of bounds.
### '300758' - Skipping patch for RA: 266.06037499999996, Dec: -29.389499999999998 as it is out of bounds.
### '303865' - Skipping patch for RA: 266.06604166666665, Dec: -29.34547222222222 as it is out of bounds.
### '304239' - Skipping patch for RA: 266.0667916666667, Dec: -29.55461111111111 as it is out of bounds.
### '343554' - Skipping patch for RA: 266.1314166666666, Dec: -29.460833333333333 as it is out of bounds.
### '348392' - Skipping patch for RA: 266.13920

#### Convert Images: FITS to NPY

In [ ]:
# 01) images_added/XXX_aligned (Size: 1162)
# image_dir1 = os.path.join(root_dir,'images_added',survey1+'_extracted')
# image_dir2 = os.path.join(root_dir,'images_added',survey2+'_extracted')

In [ ]:
# 02) images_added/XXX_aligned_cropped (Size: 1089)
# image_dir1 = os.path.join(root_dir,'images_added',survey1+'_extracted_cropped')
# image_dir2 = os.path.join(root_dir,'images_added',survey2+'_extracted_cropped')

In [ ]:
# 03) images_added/XXX_aligned_sparse_cropped (Size: 152)
# image_dir1 = os.path.join(root_dir,'images_added',survey1+'_extracted_sparse_cropped')
# image_dir2 = os.path.join(root_dir,'images_added',survey2+'_extracted_sparse_cropped')

In [28]:
# 03_01) images_added/XXX_aligned_sstgc_target (Size: 41)
image_dir1 = os.path.join(root_dir,'images_added',survey1+'_extracted_sstgc_target')
image_dir2 = os.path.join(root_dir,'images_added',survey2+'_extracted_sstgc_target')

In [ ]:
# 01) images_added/XXX_aligned
# 02) images_added/XXX_aligned_cropped
# 03) images_added/XXX_aligned_sparse_cropped
# Sort Files by Target Number
targetimagelist1 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(image_dir1, f'*.fits'))])
targetimagelist2 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(image_dir2, f'*.fits'))])

extracted_imagelist = {}
for _, row in df.iterrows():
    file_index = row["Converted_FileName"].split('_')[0]  # Extract '0001' from '0001_174042.5-292600'

    # Collect all matching files for the target index
    target_filelist = [file for file in (targetimagelist1 + targetimagelist2) if row["Converted_FileName"] in file]
    if target_filelist: extracted_imagelist[file_index] = target_filelist
len(list(extracted_imagelist))

In [30]:
# 03_01) images_added/XXX_aligned_sstgc_target
# Sort Files by Target Number
targetimagelist1 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(image_dir1, f'*.fits'))])
targetimagelist2 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(image_dir2, f'*.fits'))])

extracted_imagelist = {}
for _, row in df_sstgc.iterrows():
    file_index = row["Converted_FileName"].split('_')[0]  # Extract '0001' from '0001_174042.5-292600'

    # Collect all matching files for the target index
    target_filelist = [file for file in (targetimagelist1 + targetimagelist2) if row["Converted_FileName"] in file]
    if target_filelist: extracted_imagelist[file_index] = target_filelist
len(list(extracted_imagelist))

41

In [34]:
# Convert the Images into Numpy Array
for file_index in extracted_imagelist:
    filelist = extracted_imagelist[file_index]
    if True:#verbose_fits_to_npy: 
        print('\n'+'[[ # '+str(file_index)+' ]]')
    if True:#convert_fits_to_npy:
        for i in range(len(filelist)):
            # print(filelist[i])
            npy_converter = FitsArrayConverter(filelist[i], stacking_size=stacking_size, dir_name=dirname)
            image = npy_converter.get_image()

            # 01) images_added/XXX_aligned
            # savename = os.path.join(root_dir, 'data_added', dirname, filelist[i][0:-5].split('images_added/')[1]).replace('_extracted', '')+'.npy'
            
            # 02) images_added/XXX_aligned_cropped
            # savename = os.path.join(root_dir, 'data_added', dirname, filelist[i][0:-5].split('images_added/')[1]).replace('_extracted_cropped', '')+'.npy'

            # 03) images_added/XXX_aligned_sparse_cropped
            # savename = os.path.join(root_dir, 'data_added', dirname, filelist[i][0:-5].split('images_added/')[1]).replace('_extracted_sparse_cropped', '')+'.npy'
            
            # 03_01) images_added/XXX_aligned_sparse_cropped
            name_sub = filelist[i][0:-5].split('images_added/')[1].split('_extracted_sstgc_target/')
            savename = os.path.join(root_dir, 'data_added', dirname, name_sub[0], name_sub[1]).replace('_extracted_sparse_cropped', '')+'.npy'

            np.save(savename, image)


[[ # 534806 ]]

[[ # 584613 ]]

[[ # 618018 ]]

[[ # 619522 ]]

[[ # 619964 ]]

[[ # 646021 ]]

[[ # 653270 ]]

[[ # 670953 ]]

[[ # 673151 ]]

[[ # 679036 ]]

[[ # 689397 ]]

[[ # 696367 ]]

[[ # 711462 ]]

[[ # 716531 ]]

[[ # 718757 ]]

[[ # 719445 ]]

[[ # 721436 ]]

[[ # 722141 ]]

[[ # 726327 ]]

[[ # 728480 ]]

[[ # 732531 ]]

[[ # 738126 ]]

[[ # 760679 ]]

[[ # 761771 ]]

[[ # 769305 ]]

[[ # 770393 ]]

[[ # 771791 ]]

[[ # 772151 ]]

[[ # 772981 ]]

[[ # 773985 ]]

[[ # 782872 ]]

[[ # 786009 ]]

[[ # 790317 ]]

[[ # 797384 ]]

[[ # 799887 ]]

[[ # 801865 ]]

[[ # 803187 ]]

[[ # 803471 ]]

[[ # 806191 ]]

[[ # 817031 ]]

[[ # 817663 ]]


#### Image Stacking & Augmentation

In [7]:
# 01) images_added/XXX_aligned
# 02) images_added/XXX_aligned_cropped
# 03) images_added/XXX_aligned_sparse_cropped
npy_dir1 = os.path.join(root_dir,'data_added',dirname,survey1)
npy_dir2 = os.path.join(root_dir,'data_added',dirname,survey2)

targetnpylist1 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(npy_dir1, f'*.npy'))])
targetnpylist2 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(npy_dir2, f'*.npy'))])

npylist = {}
for _, row in df.iterrows():
    file_index = row["Converted_FileName"].split('_')[0]  # Extract '0001' from '0001_174042.5-292600'
    # Collect all matching files for the target index
    target_filelist = [file for file in (targetnpylist1 + targetnpylist2) if row["Converted_FileName"] in file]
    if target_filelist: npylist[file_index] = target_filelist
print('Num of Dataset: ',len(list(npylist)))

152


In [38]:
# 04) images_added/XXX_aligned_sparse_cropped_3ch
npy_dir1 = os.path.join(root_dir,'data_added',dirname,survey1)
npy_dir2 = os.path.join(root_dir,'data_added',dirname,survey2)

targetnpylist1 = sorted([os.path.abspath(x) for y in channels_order for x in glob.glob(os.path.join(npy_dir1, f'*_{y}.npy'))])
targetnpylist2 = sorted([os.path.abspath(x) for y in channels_order for x in glob.glob(os.path.join(npy_dir2, f'*_{y}.npy'))])
targetnpylist3 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(npy_dir2, f'*_ch1.npy'))])

npylist = {}
for _, row in df.iterrows():
    file_index = row["Converted_FileName"].split('_')[0]  # Extract '0001' from '0001_174042.5-292600'
    # Collect all matching files for the target index
    target_filelist = [file for file in (targetnpylist1 + targetnpylist2 + targetnpylist3) if row["Converted_FileName"] in file]
    if target_filelist: npylist[file_index] = target_filelist
print('Num of Dataset: ',len(list(npylist)))

Num of Dataset:  152


In [39]:
print('\n'.join(npylist['0003']), '\n')

/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/vvv/0003_174051.3-292023_H.npy
/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/vvv/0003_174051.3-292023_K.npy
/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/spitzer/0003_174051.3-292023_ch2.npy
/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/spitzer/0003_174051.3-292023_ch1.npy 



In [38]:
# 03_01) images_added/XXX_aligned_sstgc_target
npy_dir1 = os.path.join(root_dir,'data_added',dirname,survey1)
npy_dir2 = os.path.join(root_dir,'data_added',dirname,survey2)

targetnpylist1 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(npy_dir1, f'*.npy'))])
targetnpylist2 = sorted([os.path.abspath(x) for x in glob.glob(os.path.join(npy_dir2, f'*.npy'))])

npylist = {}
for _, row in df_sstgc.iterrows():
    file_index = row["Converted_FileName"].split('_')[0]  # Extract '000000' standing for 'sstgc 0000000'
    # Collect all matching files for the target index
    target_filelist = [file for file in (targetnpylist1 + targetnpylist2) if row["Converted_FileName"] in file]
    if target_filelist: npylist[file_index] = target_filelist
print('Num of Dataset: ',len(list(npylist)))
print('\n'.join(npylist['799887']), '\n')

Num of Dataset:  41
/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/sstgc_targets/vvv/799887_174724.8-281556.8_H.npy
/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/sstgc_targets/vvv/799887_174724.8-281556.8_J.npy
/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/sstgc_targets/vvv/799887_174724.8-281556.8_K.npy
/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/sstgc_targets/spitzer/799887_174724.8-281556.8_ch1.npy
/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/sstgc_targets/spitzer/799887_174724.8-281556.8_ch2.npy
/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/sstgc_targets/spitzer/799887_174724.8-281556.8_ch3.npy
/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/sstgc_targets/spitzer/799887_174724.8-281556.8_ch4.npy 



##### Stacking 6-channel Image

In [27]:
if not os.path.exists(save_dir):
    print(f'\n=== Making "{save_dir.split("/UKIDSSxIRAC/")[1]}" Directory ===\n')
    os.makedirs(save_dir)

**Options & Filename**

- `000000_X.npy`
<br/>
    ```python3
    ImageStacker(save_dir, normalization=False, logarithm=False, standardization=False,
                 channel_matching=match_output_channel_num, marking=False)    
    ```

- `000000_norm_X.npy`
<br/>
    ```python3
    ImageStacker(save_dir, normalization=True, logarithm=False, standardization=False,
                 channel_matching=match_output_channel_num, marking=True)   

- `000000_log_X.npy`
<br/>
    ```python3
    ImageStacker(save_dir, normalization=False, logarithm=True, standardization=False,
                 channel_matching=match_output_channel_num, marking=True)   
    ```

- `000000_std_X.npy`
<br/>
    ```python3
    ImageStacker(save_dir, normalization=False, logarithm=False, standardization=True,
                 channel_matching=match_output_channel_num, marking=True)   
    ```


In [30]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image

class ImageStacker:
    def __init__(self, save_dir, pop_channel=None, channels_order=None, model_name=None,
                 normalization=True, logarithm=False, standardization=False, zeroflux_sub=None,
                 replace_nan=False, replace_nan_val=None, zeroflux_sub_val=None,
                 channel_matching=None, marking=False, pre_processed=False, verbose=None):
        if pop_channel is None and channels_order is None:
            from config import channels_order as default_channels_order
            pop_channel = 'ch1'
            channels_order = default_channels_order
        elif pop_channel is not None and channels_order is None:
            raise(ValueError("'channels_order' should be specified if 'pop_channel' is given"))
        elif pop_channel is None and channels_order is not None:
            raise(ValueError("'pop_channel' should be specified if 'channels_order' is given"))
        if replace_nan and (replace_nan_val is None):
            raise (ValueError("'replace_nan_val' should be given if 'replace_nan' is True"))
        if zeroflux_sub and (zeroflux_sub_val is None):
            zeroflux_sub_val = 'median'
        if channel_matching is None:
            from config import match_output_channel_num as default_channel_matching
            channel_matching = default_channel_matching
        if verbose is None:
            from config import verbose_image_stacking as default_verbose_image_stacking
            verbose = default_verbose_image_stacking

        self.save_dir = save_dir
        self.pop_channel = pop_channel
        self.channels_order = channels_order
        self.normalization = normalization
        self.logarithm = logarithm
        self.standardization = standardization
        self.zeroflux_sub = zeroflux_sub
        self.replace_nan = replace_nan
        self.replace_nan_val = replace_nan_val
        self.zeroflux_sub_val = zeroflux_sub_val
        self.channel_matching = channel_matching
        self.marking = marking
        self.model_name = model_name
        self.pre_processed = pre_processed
        self.verbose = verbose

    def normalize_image(self, image):
        image = image.astype(np.float32)
        if np.isnan(image).any():
            min_val, max_val = np.nanmin(image), np.nanmax(image)
        else:
            min_val, max_val = np.min(image), np.max(image)
        if max_val > min_val:
            image = (image - min_val) / (max_val - min_val)
        return image

    def log_transform(self, image):
        if np.isnan(image).any():
            image = np.nan_to_num(image, nan=0.0)
        return np.log1p(image)  # log(1 + image) to avoid log(0) issues
    
    def standardize_image(self, image):
        image = image.astype(np.float32)
        if np.isnan(image).any():
            mean = np.nanmean(image)
            std = np.nanstd(image)
        else:
            mean = np.mean(image)
            std = np.std(image)
        if std > 0:
            image = (image - mean) / std
        return image
    
    def load_and_stack(self, target, target_datalist, bandlist=None):
        channel_data = {}
        for i, target_data in enumerate(target_datalist):
            if bandlist is None:
                band = os.path.splitext(os.path.split(target_data)[1].split(str(target))[1].split('_')[1])[0]
            else:
                band = bandlist[i]
            nparray = np.load(target_data)

            if self.replace_nan:
                nparray[np.isnan(nparray)] = self.replace_nan_val
            if self.zeroflux_sub:
                if self.zeroflux_sub_val == 'median':
                    self.zeroflux_sub_val = np.nanmedian(nparray)
                elif self.zeroflux_sub_val == 'mean':
                    self.zeroflux_sub_val = np.nanmean(nparray)
                elif type(self.zeroflux_sub_val) is int:
                    pass
                nparray = nparray - self.zeroflux_sub_val
                nparray[np.where(nparray < 0.0)] = 0
            if self.normalization:
                nparray = self.normalize_image(nparray)
            if self.logarithm:
                nparray = self.log_transform(nparray)
            if self.standardization:
                nparray = self.standardize_image(nparray)
            channel_data[band] = nparray

        y_data = channel_data.pop(self.pop_channel)
        X_data = np.stack([channel_data[ch] for ch in self.channels_order], axis=0)
        if self.channel_matching:
            y_data = np.repeat(y_data[np.newaxis, :, :], len(self.channels_order), axis=0)

        return X_data, y_data   # X_data.shape: (C, H, W)
    
    def mark_on_name(self):
        pre_processed =  ''
        if self.marking:
            if self.normalization:
                pre_processed = pre_processed + '_norm'
            if self.logarithm:
                pre_processed = pre_processed + '_log'
            if self.standardization:
                pre_processed = pre_processed + '_std'
            if self.zeroflux_sub:
                pre_processed = pre_processed + '_zeroflux_sub'
        return pre_processed
    
    def save_data_for_cyclegan(self):
        if self.model_name == 'cycle_gan_junyanz':
            return 'A', 'B'
        else:
            return '', ''
    
    def save_name(self, save_dir, target, ext, aug=''):
        pre_processed = self.mark_on_name()
        cyclegan_dir = self.save_data_for_cyclegan()
        filenames = [os.path.join(save_dir+cyclegan_dir[i],f'{target}_{suffix}{pre_processed}{aug}.{ext}') 
                     for i, suffix in enumerate(['X', 'y'])]
        return filenames
        
    def save_as_npy(self, target, target_datalist, save_dir=None, bandlist=None, verbose=False):
        X_data, y_data = self.load_and_stack(target, target_datalist, bandlist)
        save_dir = self.save_dir if save_dir is None else save_dir

        filename_X, filename_y = self.save_name(save_dir, target, 'npy')

        if verbose:
            print('\n'+'[[ # '+target+' ]]')
            print('X: ', np.shape(X_data), filename_X)
            print('y: ', np.shape(y_data), filename_y)

        np.save(filename_X, X_data)
        np.save(filename_y, y_data)

    def save_as_png(self, target, target_datalist, save_dir=None, verbose=False):
        X_data, y_data = self.load_and_stack(target, target_datalist)
        save_dir = self.save_dir if save_dir is None else save_dir

        filename_X, filename_y = self.save_name(save_dir, target, 'png')

        if verbose:
            print('\n'+'[[ # '+target+' ]]')
            print('X: ', np.shape(X_data), filename_X)
            print('y: ', np.shape(y_data), filename_y)

        if self.normalization: # self.normalization=False일 때 확인 필요
            X_data, y_data =  (X_data * 255).astype(np.uint8), (y_data * 255).astype(np.uint8)

        X_img = np.stack([X_data[i] for i in range(3)], axis=-1) # RGB 스택 순서 맞는지 확인 필요
        X_img = Image.fromarray(X_img)
        y_img = Image.fromarray(y_data, mode='L') # self.channel_matching=True일 때 대비 변경 필요

        X_img.save(filename_X)
        y_img.save(filename_y)

    def save_as_jpg(self, target, target_datalist, save_dir=None, verbose=False):
        X_data, y_data = self.load_and_stack(target, target_datalist)
        save_dir = self.save_dir if save_dir is None else save_dir

        filename_X, filename_y = self.save_name(save_dir, target, 'jpg')

        if verbose:
            print('\n'+'[[ # '+target+' ]]')
            print('X: ', np.shape(X_data), filename_X)
            print('y: ', np.shape(y_data), filename_y)

        if self.normalization: # self.normalization=False일 때 확인 필요
            X_data, y_data = (X_data * 255).astype(np.uint8), (y_data * 255).astype(np.uint8)
            print(np.min(X_data), np.max(X_data))

        # 데이터 확인을 위한 중간 출력
        print(f"X_data min: {np.min(X_data)}, max: {np.max(X_data)}")
        print(f"y_data min: {np.min(y_data)}, max: {np.max(y_data)}")
        
        plt.figure(figsize = (8, 3))
        plt.subplot(1, 2, 1)
        plt.imshow(X_data.transpose(1, 2, 0), cmap='gray', vmin=np.min(X_data), vmax=np.max(X_data))
        plt.axis('off')
        plt.title(f'{target}_X')

        plt.subplot(1, 2, 2)
        plt.imshow(y_data, cmap='gray')
        plt.axis('off')
        plt.title(f'{target}_y')

        print(np.shape(X_data.transpose(1, 2, 0)))

        cv2.imwrite(filename_X, X_data.transpose(1, 2, 0))
        cv2.imwrite(filename_y, y_data) # self.channel_matching=True일 때 대비 변경 필요

In [41]:
npylist['0003']

['/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/vvv/0003_174051.3-292023_H.npy',
 '/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/vvv/0003_174051.3-292023_K.npy',
 '/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/spitzer/0003_174051.3-292023_ch2.npy',
 '/home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/spitzer/0003_174051.3-292023_ch1.npy']

In [43]:
# 01) images_added/XXX_aligned
# 02) images_added/XXX_aligned_cropped
# 03) images_added/XXX_aligned_sparse_cropped
# 03_01) images_added/XXX_aligned_sstgc_target
img_stacker = ImageStacker(save_dir, normalization=True, logarithm=False, standardization=False,
                           channel_matching=match_output_channel_num, marking=True, pre_processed=False)
print(f'Save stacked data to {save_dir}')
augmentor = ImageAugmentation()

Save stacked data to /home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/merged_3ch


In [44]:
# 04) images_added/XXX_aligned_sparse_cropped_3ch
img_stacker = ImageStacker(save_dir, normalization=True, logarithm=False, standardization=False,
                           channel_matching=match_output_channel_num, marking=True, pre_processed=False,
                           channels_order=channels_order, pop_channel='ch1')
print(f'Save stacked data to {save_dir}')
augmentor = ImageAugmentation()

Save stacked data to /home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/merged_3ch


In [46]:
if True:#save_stacked_image:
    for file_index in list(npylist):
        target_datalist = npylist[file_index]
        target_datalist.sort()

        import re
        bandlist = []
        for filepath in target_datalist:
            match = re.search(r'_([a-zA-Z0-9]+)\.npy$', filepath)
            if match:
                bandlist.append(match.group(1))
        file_suffix_name ='_'.join(os.path.basename(target_datalist[0]).split('_')[0:2])   # print(file_suffix_name)
        img_stacker.save_as_npy(file_suffix_name, target_datalist, bandlist=bandlist, verbose=verbose_image_stacking)
        
        if len(augmentation) > 0:
            filenames = img_stacker.save_name(save_dir,file_suffix_name,'npy')
            X_data, y_data = np.load(filenames[0]), np.load(filenames[1])

            for aug in augmentation:
                aug_data_X = getattr(augmentor, aug)(X_data)
                aug_data_y = getattr(augmentor, aug)(y_data)
                
                np.save(os.path.splitext(filenames[0])[0]+f'_{aug}.npy', aug_data_X)
                np.save(os.path.splitext(filenames[1])[0]+f'_{aug}.npy', aug_data_y)
                
                if verbose_augmentation:
                    print(os.path.splitext(filenames[0])[0]+f'_{aug}.npy')
                    print(os.path.splitext(filenames[1])[0]+f'_{aug}.npy')


[[ # 0003_174051.3-292023 ]]
X:  (3, 174, 174) /home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/merged_3ch/0003_174051.3-292023_X_norm.npy
y:  (3, 174, 174) /home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/merged_3ch/0003_174051.3-292023_y_norm.npy

[[ # 0004_174059.0-291034 ]]
X:  (3, 174, 174) /home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/merged_3ch/0004_174059.0-291034_X_norm.npy
y:  (3, 174, 174) /home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/merged_3ch/0004_174059.0-291034_y_norm.npy

[[ # 0006_174103.7-291404 ]]
X:  (3, 174, 174) /home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/merged_3ch/0006_174103.7-291404_X_norm.npy
y:  (3, 174, 174) /home/jiwon/UKIDSSxIRAC/data_added/registrated_sparse_cropped_gc_180x180_extraction/merged_3ch/0006_174103.7-291404_y_norm.npy

[[ # 0007_174108.7-291630 ]]
X:  (3, 17

In [47]:
source_list = []
for file_index in list(npylist):
    target_datalist = npylist[file_index]
    file_suffix_name ='_'.join(os.path.basename(target_datalist[0]).split('_')[0:2])
    source_list.append(file_suffix_name)

In [48]:
source_list.sort()
print('\n'.join(source_list), '\n')
print()
print(len(source_list))

0003_174051.3-292023
0004_174059.0-291034
0006_174103.7-291404
0007_174108.7-291630
0008_174111.3-291455
0011_174119.2-291704
0012_174122.4-291102
0013_174122.9-292653
0015_174124.6-291249
0016_174124.9-291853
0017_174126.8-291608
0018_174127.9-292152
0019_174129.2-291145
0020_174129.9-291150
0021_174130.1-291225
0023_174131.8-291917
0024_174132.7-292043
0025_174134.0-291605
0026_174135.2-292330
0028_174135.7-292234
0030_174138.1-285646
0031_174138.1-291159
0032_174142.8-291440
0035_174151.6-290557
0036_174151.8-291949
0037_174155.3-291215
0038_174159.0-291325
0041_174203.5-292136
0042_174208.3-292116
0043_174208.6-285537
0044_174210.6-292157
0045_174215.0-290043
0046_174215.7-290557
0047_174215.8-290542
0048_174217.7-292522
0049_174217.9-291818
0050_174219.7-292422
0051_174219.8-293516
0052_174220.4-292432
0053_174220.5-285321
0054_174220.5-293243
0055_174222.9-292855
0057_174224.1-290125
0059_174226.8-292217
0060_174227.2-291137
0061_174227.7-292419
0062_174228.7-292526
0063_174229.8

In [49]:
output_file = "source_list_VVV_IRAC_sparse_cropped_3ch.txt"

# 리스트를 텍스트 파일로 저장
with open(output_file, "w") as f:
    f.write("\n".join(source_list))